1. Load data and model

In [29]:
import torch

if torch.cuda.is_available():
    print("GPU is available!")
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    print("GPU is not available. Check your setup.")


GPU is available!
Using GPU: NVIDIA GeForce RTX 4080 Laptop GPU


In [30]:
# Import required libraries
import os
from datasets import Dataset

# Function to read IOB files and split into paragraphs
def read_iob_file(file_path):
    """Read IOB file and convert to token-label samples"""
    examples = []
    words, labels = [], []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                token, label = line.split("\t")
                words.append(token)
                labels.append(label.strip())
            else:
                # End of a sample
                if words and labels:
                    examples.append({"tokens": words, "labels": labels})
                    words, labels = [], []
    # Add the last example if file doesn't end with a blank line
    if words and labels:
        examples.append({"tokens": words, "labels": labels})
    return examples



In [31]:
# Load datasets using the updated function
train_path = 'C:\\S24-25\\TxM\\dataset\\train.tsv'
val_path = 'C:\\S24-25\\TxM\\dataset\\val_gold.tsv'
test_path = 'C:\\S24-25\\TxM\\dataset\\test_gold.tsv'
train_data = read_iob_file(train_path)
val_data = read_iob_file(val_path)
test_data = read_iob_file(test_path)

print(f"Train samples: {len(train_data)}, Val samples: {len(val_data)}, Test samples: {len(test_data)}")


Train samples: 998, Val samples: 124, Test samples: 126


In [43]:
from transformers import AutoTokenizer
# Load model directly
# Use a pipeline as a high-level helper
from transformers import pipeline
model_checkpoint="emilyalsentzer/Bio_ClinicalBERT"
pipe = pipeline("fill-mask", model="emilyalsentzer/Bio_ClinicalBERT")
# Define label mappings
label_list = ["O", "B-ADR", "I-ADR", "B-DRU", "I-DRU", "B-DIS", "I-DIS", "B-SYM", "I-SYM"]
label_to_id = {label: i for i, label in enumerate(label_list)}
id_to_label = {i: label for label, i in label_to_id.items()}

def tokenize_and_align_labels(examples):
    tokenized_inputs = pipe.tokenizer(
        examples["tokens"], 
        is_split_into_words=True,
        padding="max_length",      # or True, depending on your preference
        truncation=True,          # enable truncation
        max_length=512,           # explicitly set maximum sequence length 
        return_overflowing_tokens=False,
    )
    labels = []
    for i, label in enumerate(examples["labels"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        aligned_labels = []
        previous_word_idx = None
        for word_idx in word_ids:
            if word_idx is None:
                aligned_labels.append(-100)  # Ignore padding
            elif word_idx != previous_word_idx:
                aligned_labels.append(label_to_id[label[word_idx]])
            else:
                aligned_labels.append(label_to_id[label[word_idx]])
            previous_word_idx = word_idx
        labels.append(aligned_labels)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# Tokenize datasets
train_dataset = Dataset.from_list(train_data).map(tokenize_and_align_labels, batched=True)
test_dataset = Dataset.from_list(test_data).map(tokenize_and_align_labels, batched=True)
val_dataset = Dataset.from_list(val_data).map(tokenize_and_align_labels, batched=True)


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Map: 100%|██████████| 998/998 [00:00<00:00, 2624.21 examples/s]

Map: 100%|██████████| 126/126 [00:00<00:00, 2342.01 examples/s]

Map: 100%|██████████| 124/124 [00:00<00:00, 2486.73 examples/s]


In [44]:
from transformers import AutoModelForTokenClassification


# Load the pre-trained BERT model

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    num_labels=len(label_list),
    id2label=id_to_label,
    label2id=label_to_id
)


Some weights of BertForTokenClassification were not initialized from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


2. Train and evaluate with default parameters on the test set

In [45]:
from transformers import TrainingArguments

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    load_best_model_at_end=True,
    logging_dir="./bio_bert_logs",
    logging_steps=10,
)


In [48]:
from transformers import Trainer
from evaluate import load
import torch

# Load metric for evaluation
metric = load("seqeval")

def compute_metrics(predictions):
    predictions, labels = predictions
    predictions = torch.argmax(torch.tensor(predictions), dim=2)
    
    # Convert predictions and labels to CPU and then to plain Python integers
    predictions = predictions.cpu().numpy()
    
    true_labels = [
        [id_to_label[label] for label in label_seq if label != -100] 
        for label_seq in labels
    ]
    true_predictions = [
        [id_to_label[int(p)] for (p, l) in zip(prediction, label_seq) if l != -100]  # Convert tensor to int
        for prediction, label_seq in zip(predictions, labels)
    ]
    return metric.compute(predictions=true_predictions, references=true_labels)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=pipe.tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()


  5%|▌         | 10/189 [00:08<01:20,  2.23it/s]

{'loss': 0.217, 'grad_norm': 1.288893461227417, 'learning_rate': 1.8941798941798943e-05, 'epoch': 0.16}


 11%|█         | 20/189 [00:11<01:03,  2.68it/s]

{'loss': 0.2185, 'grad_norm': 1.2395459413528442, 'learning_rate': 1.7883597883597884e-05, 'epoch': 0.32}


 16%|█▌        | 30/189 [00:15<00:58,  2.71it/s]

...

100%|██████████| 189/189 [01:18<00:00,  3.28it/s]c:\Users\HikaLipp\miniconda3\envs\TxM\Lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))

100%|██████████| 189/189 [01:20<00:00,  3.28it/s]

{'eval_loss': 0.22910471260547638, 'eval_ADR': {'precision': 0.4652482269503546, 'recall': 0.5403624382207578, 'f1': 0.5, 'number': 607}, 'eval_DIS': {'precision': 0.21739130434782608, 'recall': 0.08620689655172414, 'f1': 0.12345679012345678, 'number': 58}, 'eval_DRU': {'precision': 0.8636363636363636, 'recall': 0.8850931677018633, 'f1': 0.8742331288343559, 'number': 322}, 'eval_SYM': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 50}, 'eval_overall_precision': 0.5841209829867675, 'eval_overall_recall': 0.5959498553519769, 'eval_overall_f1': 0.5899761336515513, 'eval_overall_accuracy': 0.9297758515609128, 'eval_runtime': 1.0596, 'eval_samples_per_second': 118.912, 'eval_steps_per_second': 7.55, 'epoch': 3.0}


100%|██████████| 189/189 [01:22<00:00,  2.30it/s]

{'train_runtime': 82.0785, 'train_samples_per_second': 36.477, 'train_steps_per_second': 2.303, 'train_loss': 0.16333529431983906, 'epoch': 3.0}


TrainOutput(global_step=189, training_loss=0.16333529431983906, metrics={'train_runtime': 82.0785, 'train_samples_per_second': 36.477, 'train_steps_per_second': 2.303, 'total_flos': 782372000176128.0, 'train_loss': 0.16333529431983906, 'epoch': 3.0})

In [49]:
# Evaluate the model
results = trainer.evaluate()
print("Evaluation Results:", results)


 88%|████████▊ | 7/8 [00:00<00:00,  9.28it/s]c:\Users\HikaLipp\miniconda3\envs\TxM\Lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
100%|██████████| 8/8 [00:00<00:00,  8.33it/s]

Evaluation Results: {'eval_loss': 0.22910471260547638, 'eval_ADR': {'precision': 0.4652482269503546, 'recall': 0.5403624382207578, 'f1': 0.5, 'number': 607}, 'eval_DIS': {'precision': 0.21739130434782608, 'recall': 0.08620689655172414, 'f1': 0.12345679012345678, 'number': 58}, 'eval_DRU': {'precision': 0.8636363636363636, 'recall': 0.8850931677018633, 'f1': 0.8742331288343559, 'number': 322}, 'eval_SYM': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 50}, 'eval_overall_precision': 0.5841209829867675, 'eval_overall_recall': 0.5959498553519769, 'eval_overall_f1': 0.5899761336515513, 'eval_overall_accuracy': 0.9297758515609128, 'eval_runtime': 1.3838, 'eval_samples_per_second': 91.051, 'eval_steps_per_second': 5.781, 'epoch': 3.0}


3. Train and test on val set

4. HPO